In [1]:
import pandas as pd, numpy as np, matplotlib.pyplot as plt
import utm
import matplotlib.ticker as ticker
from itertools import groupby
import os, os.path
import seaborn as sns
from datetime import datetime
from sklearn.cluster import DBSCAN
from sklearn import metrics
from sklearn import preprocessing
import plotly.express as px
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import davies_bouldin_score

In [ ]:
def convert_long_lat_to_XY(input_lon,input_lat):
  x, y, zone, ut = utm.from_latlon(input_lat, input_lon)
  return x,y
#To convert time into seconds
def get_sec(time_str):
    """Get Seconds from time."""
    h, m, s = time_str.split(':')
    return int(h) * 3600 + int(m) * 60 + int(s)

In [18]:
directory_path = 'BrightKite_converted/'
file_list = os.listdir(directory_path)

In [19]:
min_lat = []
max_lat = []
min_long = []
max_long = []
for file_name in file_list:
    df = pd.read_csv('BrightKite_converted/'+file_name, encoding='utf-8')
    coords = df[['Latitude', 'Longitude']].to_numpy()
    lats_xy = coords[:,0]
    longs_xy = coords[:,1]
    #Finding minimum and maximum points from user data
    lats_index_min = np.argmin(lats_xy)
    lats_index_max = np.argmax(lats_xy)
    longs_index_min = np.argmin(longs_xy)
    longs_index_max = np.argmax(longs_xy)
    
    min_lat.append(lats_xy[lats_index_min])
    max_lat.append(lats_xy[lats_index_max])
    min_long.append(longs_xy[longs_index_min])
    max_long.append(longs_xy[longs_index_max])
    
minimum_lat = min(min_lat)
maximum_lat = max(max_lat)
minimum_long = min(min_long)
maximum_long = max(max_long)

In [20]:
print('Min Latitude: ',minimum_lat)
print('Max latitude: ',maximum_lat)
print('Min Longitude: ',minimum_long)
print('Max longitude: ',maximum_long)

Min Latitude:  0.0
Max latitude:  9999985.500660034
Min Longitude:  166021.4431793313
Max longitude:  831916.7581531521


In [ ]:
for file_name in file_list:
    df = pd.read_csv('Brightkite/'+file_name, encoding='utf-8')
    df = df.iloc[:,2:]
    df = df.drop(columns='Date')
    coords = df[['Latitude', 'Longitude']].to_numpy()
    time_user = df[['Time']].to_numpy()
    try:
        si = time_user.shape
        time_size = si[0]
        a = time_user.astype(str)

        #To store time in hours
        time_conv = []
        for i in range(time_size):
            new = np.array_str(a[i])
            s1 = new.replace('[','')
            s2 = s1.replace(']','')
            s3 = s2[1:]
            s4 = s3[:-1]
            con = get_sec(s4)
            time_conv.append(con)
        user_time_sec = np.array(time_conv)
        user_time_hour = user_time_sec/3600

        lats = coords[:,0]
        longs = coords[:,1]
        size1 = lats.shape
        size_coords = size1[0]
        coords_XY = []
        for i in range(size_coords):
            p_xy = convert_long_lat_to_XY(longs[i],lats[i])
            coords_XY.append(p_xy)
        coords_conv_xy = np.array(coords_XY)
        lats_xy = coords_conv_xy[:,1]
        longs_xy = coords_conv_xy[:,0]

        df_new = pd.DataFrame({'Latitude': lats_xy, 'Longitude': longs_xy, 'Time': user_time_hour})
        file_path = ("BrightKite_converted/"+file_name)

        df_new.to_csv(file_path, index=False)
        
        pass
    except Exception as e:
        continue

In [2]:
directory_path = "BrightKite_converted/"

file_list = [file for file in os.listdir(directory_path) if file.endswith('.csv')]

lengths = []

In [11]:
small = 0
moderate = 0
large = 0
enormous = 0
mod_files = []
small_files = []
for file_name in file_list:
    df = pd.read_csv(os.path.join(directory_path, file_name))
    length = len(df)
    
    if length < 1000:
        small += 1
        if small < 1000 and length>500:
            small_files.append(file_name)
        
    elif length > 1000 and length <10000:
        moderate += 1
        
    elif length >10000 and length<100000:
        large += 1
        
    else:
        enormous += 1
    
    lengths.append(length)
    
print('Small category users: ',small)
print('Moderate category users: ',moderate)
print('Large category users: ',large)
print('Enormous category users: ',enormous)

Small category users:  50295
Moderate category users:  1093
Large category users:  0
Enormous category users:  0


In [6]:
with open('my_list.txt', 'w') as f:
    for item in mod_files:
        f.write('%s\n' % item)

In [16]:
df10 = pd.read_csv('BrightKite/user_10025.csv', encoding='utf-8')
df10 = df10.iloc[:,2:]
df10 = df10.drop(columns='Date')
df10.to_csv('User_10025', index=False)